In [1]:
from src.model.data_module import get_random_batch_data
batch_data = get_random_batch_data()





main path: /Users/mengkjin/workspace/learndl
src.INSTANCE_RECORD can be accessed to check ['trainer', 'account', 'factor']


In [2]:
import torch
import torch.nn as nn
from torch import Tensor

class StockWiseTranformer(nn.Module):
    def __init__(self , input_dim , hidden_dim = 4 , ffn_dim = 16 , num_heads = 8 , encoder_layers = 2 , dropout=0.0):
        super().__init__()
        self.fc_att = TimeWiseAttention(hidden_dim,hidden_dim)
        enc_layer  = nn.TransformerEncoderLayer(hidden_dim, num_heads, dim_feedforward=ffn_dim , dropout=dropout , batch_first=True)
        self.trans = nn.TransformerEncoder(enc_layer , encoder_layers)

    def forward(self, x : Tensor , pad_mask = None) -> Tensor:
        if x.isnan().any():
            pad_mask = self.pad_mask_nan(x) if pad_mask is None else (self.pad_mask_nan(x) + pad_mask) > 0
            x = x.nan_to_num()
        if x.dim() != 2: x = self.fc_att(x)
        x = self.trans(x.unsqueeze(0) , src_key_padding_mask = pad_mask)
        return x.squeeze(0)

torch.Size([10, 30, 6])